In [1]:
from pathlib import Path

import pandas as pd
from lightning import pytorch as pl
import ray
from ray import tune
from ray.train import CheckpointConfig, RunConfig, ScalingConfig
from ray.train.lightning import (RayDDPStrategy, RayLightningEnvironment,
                                 RayTrainReportCallback, prepare_trainer)
from ray.train.torch import TorchTrainer
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import FIFOScheduler

from chemprop import data, featurizers, models, nn

/home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-12 00:18:14,722	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-12 00:18:15,460	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-12 00:18:16,578	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
chemprop_dir = Path.cwd().parent
train_path = "train_data_5ht1b.csv"
test_path = "test_data_5ht1b.csv"
num_workers = 0 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'smiles' # name of the column containing SMILES strings
target_columns = ['value'] # list of names of the columns containing targets

hpopt_save_dir = Path.cwd() / "hpopt_5ht1b_zinc" # directory to save hyperopt results
hpopt_save_dir.mkdir(exist_ok=True)

In [3]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [4]:
# Extract SMILES and target values
train_smis = df_train[smiles_column].values
train_ys = df_train[target_columns].values

test_smis = df_test[smiles_column].values
test_ys = df_test[target_columns].values

# Convert data to MoleculeDatapoint format
train_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(train_smis, train_ys)]
test_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(test_smis, test_ys)]


In [5]:
# Initialize featurizer
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

# Create training dataset
train_dset = data.MoleculeDataset(train_data, featurizer)
scaler = train_dset.normalize_targets()

# Split validation set from training data (80% train, 20% validation)
split_idx = int(len(train_data) * 0.8)
val_data = train_data[split_idx:]
train_data = train_data[:split_idx]

# Create validation and test datasets
val_dset = data.MoleculeDataset(val_data, featurizer)
val_dset.normalize_targets(scaler)

test_dset = data.MoleculeDataset(test_data, featurizer)

In [6]:
def train_model(config, train_dset, val_dset, num_workers, scaler):

    # config is a dictionary containing hyperparameters used for the trial
    depth = int(config["depth"])
    ffn_hidden_dim = int(config["ffn_hidden_dim"])
    ffn_num_layers = int(config["ffn_num_layers"])
    message_hidden_dim = int(config["message_hidden_dim"])

    train_loader = data.build_dataloader(train_dset, num_workers=num_workers, shuffle=True)
    val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)

    mp = nn.BondMessagePassing(d_h=message_hidden_dim, depth=depth)
    agg = nn.MeanAggregation()
    output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
    ffn = nn.RegressionFFN(output_transform=output_transform, input_dim=message_hidden_dim, hidden_dim=ffn_hidden_dim, n_layers=ffn_num_layers)
    batch_norm = True
    metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()]
    model = models.MPNN(mp, agg, ffn, batch_norm, metric_list)

    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        max_epochs=20, # number of epochs to train for
        # below are needed for Ray and Lightning integration
        strategy=RayDDPStrategy(),
        callbacks=[RayTrainReportCallback()],
        plugins=[RayLightningEnvironment()],
    )

    trainer = prepare_trainer(trainer)
    trainer.fit(model, train_loader, val_loader)

In [7]:
search_space = {
    "depth": tune.qrandint(lower=2, upper=6, q=1),
    "ffn_hidden_dim": tune.qrandint(lower=300, upper=2400, q=100),
    "ffn_num_layers": tune.qrandint(lower=1, upper=3, q=1),
    "message_hidden_dim": tune.qrandint(lower=300, upper=2400, q=100),
}

In [8]:
ray.init()

scheduler = FIFOScheduler()

# Scaling config controls the resources used by Ray
scaling_config = ScalingConfig(
    num_workers=1,
    use_gpu=False, # change to True if you want to use GPU
)

# Checkpoint config controls the checkpointing behavior of Ray
checkpoint_config = CheckpointConfig(
    num_to_keep=1, # number of checkpoints to keep
    checkpoint_score_attribute="val_loss", # Save the checkpoint based on this metric
    checkpoint_score_order="min", # Save the checkpoint with the lowest metric value
)

run_config = RunConfig(
    checkpoint_config=checkpoint_config,
    storage_path=hpopt_save_dir / "ray_results", # directory to save the results
)

ray_trainer = TorchTrainer(
    lambda config: train_model(
        config, train_dset, val_dset, num_workers, scaler
    ),
    scaling_config=scaling_config,
    run_config=run_config,
)

search_alg = HyperOptSearch(
    n_initial_points=1, # number of random evaluations before tree parzen estimators
    random_state_seed=42,
)

# OptunaSearch is another search algorithm that can be used
# search_alg = OptunaSearch()

tune_config = tune.TuneConfig(
    metric="val_loss",
    mode="min",
    num_samples=2, # number of trials to run
    scheduler=scheduler,
    search_alg=search_alg,
    trial_dirname_creator=lambda trial: str(trial.trial_id), # shorten filepaths

)

tuner = tune.Tuner(
    ray_trainer,
    param_space={
        "train_loop_config": search_space,
    },
    tune_config=tune_config,
)

# Start the hyperparameter search
results = tuner.fit()

(TorchTrainer pid=2724912) Started distributed worker processes: 
(TorchTrainer pid=2724912) - (node_id=0194c22a06ea2b7a23c8f0f411a980ab368c58c49aab1c08f33a3da5, ip=10.233.0.55, pid=2725150) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=2725150) Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=2725150) /home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/ray/train/lightning/_lightning_utils.py:262: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
(RayTrainWorker pid=2725150) `get_trial_name` is deprecated because the concept of a `Trial` will soon be removed in Ray Train.Ray Train will no longer assume that it's running within a Ray Tune `Trial` in the future. See this issue for more context and migration options: https://github.com/ray-project/ray/issues/49454. Disable these warnings by setting the envi

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]


(RayTrainWorker pid=2725150) /home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/lightning/pytorch/core/saving.py:363: Skipping 'metrics' parameter because it is not possible to safely dump to YAML.
(RayTrainWorker pid=2725150) /home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:00<00:00,  2.91it/s]
Training: |          | 0/? [00:00<?, ?it/s]                                
Epoch 0:  64%|██████▍   | 18/28 [00:06<00:03,  2.88it/s, v_num=3.41e+7, train_loss_step=0.872]


(TorchTrainer pid=2725149) Started distributed worker processes: 
(TorchTrainer pid=2725149) - (node_id=0194c22a06ea2b7a23c8f0f411a980ab368c58c49aab1c08f33a3da5, ip=10.233.0.55, pid=2725548) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=2725548) Setting up process group for: env:// [rank=0, world_size=1]


...

(RayTrainWorker pid=2725548) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_5ht1b_zinc/ray_results/TorchTrainer_2025-03-12_00-19-26/6ccfe64d/checkpoint_000019)
(RayTrainWorker pid=2725548) `Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 18: 100%|██████████| 28/28 [00:09<00:00,  3.03it/s, v_num=3.41e+7, train_loss_step=0.374, val_loss=0.393, train_loss_epoch=0.472]
Validation: |          | 0/? [00:00<?, ?it/s]
(RayTrainWorker pid=2725150) 
Validation DataLoader 0:   0%|          | 0/6 [00:00<?, ?it/s]
(RayTrainWorker pid=2725150) 
Validation DataLoader 0:  17%|█▋        | 1/6 [00:00<00:00,  5.38it/s]
(RayTrainWorker pid=2725150) 
Epoch 18:  89%|████████▉ | 25/28 [00:08<00:01,  2.96it/s, v_num=3.41e+7, train_loss_step=0.562, val_loss=0.393, train_loss_epoch=0.472] [repeated 13x across cluster]
(RayTrainWorker pid=2725150) 
Validation DataLoader 0:  50%|█████     | 3/6 [00:00<00:00,  5.02it/s]
(RayTrainWorker pid=2725150) 
Validation DataLoader 0:  67%|██████▋   | 4/6 [00:00<00:00,  4.88it/s]
(RayTrainWorker pid=2725150) 
Validation DataLoader 0:  83%|████████▎ | 5/6 [00:01<00:00,  4.75it/s]
(RayTrainWorker pid=2725150) 
Epoch 19:   0%|          | 0/28 [00:00<?, ?it/s, v_num=3.41e+7, train_loss_step=0.374, val_loss

(RayTrainWorker pid=2725150) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_5ht1b_zinc/ray_results/TorchTrainer_2025-03-12_00-19-26/6f0157cc/checkpoint_000018)


Epoch 19: 100%|██████████| 28/28 [00:07<00:00,  3.63it/s, v_num=3.41e+7, train_loss_step=0.376, val_loss=0.402, train_loss_epoch=0.455]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0:   0%|          | 0/6 [00:00<?, ?it/s]
(RayTrainWorker pid=2725150) 
Validation DataLoader 0:  17%|█▋        | 1/6 [00:00<00:00,  5.42it/s]
(RayTrainWorker pid=2725150) 
Validation DataLoader 0:  33%|███▎      | 2/6 [00:00<00:00,  4.92it/s]
(RayTrainWorker pid=2725150) 
Validation DataLoader 0:  50%|█████     | 3/6 [00:00<00:00,  4.96it/s]
(RayTrainWorker pid=2725150) 
Validation DataLoader 0:  67%|██████▋   | 4/6 [00:00<00:00,  4.94it/s]
(RayTrainWorker pid=2725150) 
Validation DataLoader 0:  83%|████████▎ | 5/6 [00:01<00:00,  4.93it/s]
(RayTrainWorker pid=2725150) 
Epoch 19:  89%|████████▉ | 25/28 [00:06<00:00,  3.61it/s, v_num=3.41e+7, train_loss_step=0.580, val_loss=0.402, train_loss_epoch=0.455] [repeated 10x across cluster]


(RayTrainWorker pid=2725150) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_5ht1b_zinc/ray_results/TorchTrainer_2025-03-12_00-19-26/6f0157cc/checkpoint_000019)
(RayTrainWorker pid=2725150) `Trainer.fit` stopped: `max_epochs=20` reached.
2025-03-12 00:23:49,043	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_5ht1b_zinc/ray_results/TorchTrainer_2025-03-12_00-19-26' in 0.0063s.
2025-03-12 00:23:49,048	INFO tune.py:1041 -- Total run time: 262.56 seconds (262.42 seconds for the tuning loop).


In [9]:
results

ResultGrid<[
  Result(
    metrics={'train_loss': 0.4388100206851959, 'train_loss_step': 0.3762468099594116, 'val/rmse': 0.6160438656806946, 'val/mae': 0.45948243141174316, 'val_loss': 0.3795100152492523, 'train_loss_epoch': 0.4388100206851959, 'epoch': 19, 'step': 560},
    path='/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_5ht1b_zinc/ray_results/TorchTrainer_2025-03-12_00-19-26/6f0157cc',
    filesystem='local',
    checkpoint=Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_5ht1b_zinc/ray_results/TorchTrainer_2025-03-12_00-19-26/6f0157cc/checkpoint_000019)
  ),
  Result(
    metrics={'train_loss': 0.3525916337966919, 'train_loss_step': 0.30561158061027527, 'val/rmse': 0.538872241973877, 'val/mae': 0.42712679505348206, 'val_loss': 0.29038330912590027, 'train_loss_epoch': 0.3525916337966919, 'epoch': 19, 'step': 560},
    path='/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_5ht1b_zinc/ray_results/TorchTrainer_202

In [10]:
result_df = results.get_dataframe()
result_df

,train_loss,train_loss_step,val/rmse,val/mae,val_loss,train_loss_epoch,epoch,step,timestamp,checkpoint_dir_name,...,pid,hostname,node_ip,time_since_restore,iterations_since_restore,config/train_loop_config/depth,config/train_loop_config/ffn_hidden_dim,config/train_loop_config/ffn_num_layers,config/train_loop_config/message_hidden_dim,logdir
0,0.438810,0.376247,0.616044,0.459482,0.379510,0.438810,19,560,1741735427,checkpoint_000019,...,2724912,cpusrv45.scidom.de,10.233.0.55,250.919009,20,2,2000,2,500,6f0157cc
1,0.352592,0.305612,0.538872,0.427127,0.290383,0.352592,19,560,1741735412,checkpoint_000019,...,2725149,cpusrv45.scidom.de,10.233.0.55,225.522838,20,2,2200,2,400,6ccfe64d


In [11]:
# best configuration
best_result = results.get_best_result()
best_config = best_result.config
best_config['train_loop_config']

{'depth': 2,
 'ffn_hidden_dim': 2200,
 'ffn_num_layers': 2,
 'message_hidden_dim': 400}

In [12]:
# best model checkpoint path
best_result = results.get_best_result()
best_checkpoint_path = Path(best_result.checkpoint.path) / "checkpoint.ckpt"
print(f"Best model checkpoint path: {best_checkpoint_path}")

Best model checkpoint path: /ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_5ht1b_zinc/ray_results/TorchTrainer_2025-03-12_00-19-26/6ccfe64d/checkpoint_000019/checkpoint.ckpt


In [13]:
mpnn = models.MPNN.load_from_checkpoint(best_checkpoint_path)
mpnn

MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=86, out_features=400, bias=False)
    (W_h): Linear(in_features=400, out_features=400, bias=False)
    (W_o): Linear(in_features=472, out_features=400, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): MeanAggregation()
  (bn): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=400, out_features=2200, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=2200, out_features=2200, bias=True)
      )
      (2): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=2200, out_features=1, bias=True)
      )
    )
    (criterion): MSE(task_weights=[[1.0]

In [14]:
import torch
test_loader = data.build_dataloader(test_dset, shuffle=False)
with torch.inference_mode():
    trainer = pl.Trainer(
        logger=None,
        enable_progress_bar=True,
        accelerator="cpu",
        devices=1
    )
    test_preds = trainer.predict(mpnn, test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
SLURM auto-requeueing enabled. Setting signal handlers.
/home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 11.48it/s]


In [15]:
import numpy as np
test_preds = np.concatenate(test_preds, axis=0)
df_test['preds'] = test_preds
df_test

,smiles,value,preds
0,COc1ccc(C(=O)Nc2ccccc2C(=O)Nc2ccccn2)cc1,9.8,9.118670
1,CCc1noc(CC)c1CNC(=O)N[C@H]1CCc2c(F)cccc21,9.5,7.903823
2,CC(C)C[C@@H](C[NH2+]C1CC1)C[C@@H]1COc2ccccc21,7.3,8.024309
3,CC[NH+]1CCc2c(cccc2-n2nnc(C(=O)NC3CC3)c2C)C1,6.5,8.062380
4,Cc1ccc(N2C[C@H](C(=O)N3CCC[C@@H](C(=O)[O-])C3)...,9.7,10.343315
...,...,...,...
193,Cc1[nH]ncc1CCCNc1ccc(Br)cc1[N+](=O)[O-],6.9,7.192903
194,C[C@@H](NC(=O)c1ccccc1NC(=O)Cn1ccc([N+](=O)[O-...,10.4,8.987608
195,COc1cccc(-c2cc(C(=O)N3CCC[C@@H]3c3cc(C(C)C)no3...,10.3,8.561325
196,CC(C)c1ccc2c(c1)[C@@H]([NH2+][C@@H]1CCO[C@]3(C...,9.4,8.666800


In [16]:
from sklearn.metrics import r2_score, mean_squared_error

# Get true values (ground truth) and predictions
y_true = df_test['value'].values  # True target values
y_pred = df_test['preds'].values  # Predicted values

# Calculate metrics
r2 = r2_score(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)

print(f"R² Score: {r2:.4f}")
print(f"MSE: {mse:.4f}")

R² Score: 0.1920
MSE: 1.3791
